In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
from google.colab import files
token = files.upload()

In [0]:
import os
os.chdir(r'/content/drive/My Drive/MAIS 202 Assignments/Assignment 4')

In [0]:
import os
os.chdir(r'/content/drive/My Drive/')

In [0]:
!mkdir /root/.kaggle/
!cp kaggle.json /root/.kaggle/ 
!kaggle competitions download -c winter2020-mais-202

cp: cannot stat 'kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
!unzip -o 'train_images.npy.zip' 
!unzip -o 'test_images.npy.zip'

unzip:  cannot find or open train_images.npy.zip, train_images.npy.zip.zip or train_images.npy.zip.ZIP.
unzip:  cannot find or open test_images.npy.zip, test_images.npy.zip.zip or test_images.npy.zip.ZIP.


In [0]:
# !pip install numpy

import keras
from keras import optimizers
from keras import regularizers
from keras.layers import BatchNormalization
from keras.datasets import mnist
from keras.applications import VGG16, VGG19, InceptionResNetV2, Xception, NASNetLarge
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.utils import shuffle

import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import csv
import glob
import random
import math

import tensorflow as tf


In [0]:
!tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt

TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 0 files
Detected 0 issues that require attention
--------------------------------------------------------------------------------


Make sure to read the detailed log 'report.txt'



In [0]:
!pwd

/content/drive/My Drive


# Preprocessing

In [0]:
train_images = np.load('train_images.npy')
test_images = np.load('test_images.npy')

#reshape into 2 dimensions
X_train = train_images.reshape(len(train_images),-1)
X_train = X_train

X_test = test_images.reshape(len(test_images),-1)
X_test = X_test
# print(train_images)

# print(X_train)
# print(len(train_images))

y_train = pd.read_csv('train_labels.csv')
y_train = y_train["label"]

#shuffle dataset
X_train, y_train = shuffle(X_train, y_train)

In [0]:
y_train[:10]

24965    4
15158    3
3565     3
7997     6
5003     0
1478     8
2802     0
31605    2
36691    4
15853    2
Name: label, dtype: int64

In [0]:
y = y_train.to_numpy()

In [0]:
y_train.shape

(50000,)

In [0]:
X_train.shape

(40000, 784)

In [0]:
y[:10]

array([4, 3, 3, 6, 0, 8, 0, 2, 4, 2])

In [0]:
y.shape

(50000,)

In [0]:
X_train.shape

(50000, 784)

### Random Forest

In [0]:
rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)
print('Training accuracy: ', rfc.score(X_train, y_train))

Training accuracy:  1.0


In [0]:
y = rfc.predict(X_test)
print(y)

[9 5 2 ... 3 8 6]


In [0]:
with open('submission.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['ID', 'label'])

    for c, row in enumerate(y):
      writer.writerow((c, row))

# CNN

In [0]:
import os
os.chdir(r'/content/drive/My Drive/McGill/Winter 2020/MAIS 202 Bootcamp/Assignment/Assignment 4')

In [0]:
X_train = np.load('train_images.npy')
X_test = np.load('test_images.npy')

y_train = pd.read_csv('train_labels.csv')
y_train = y_train["label"]
y_train = y_train.to_numpy()

In [0]:
X_train = X_train.reshape(50000, 28, 28, 1)
X_test = X_test.reshape(20000, 28, 28, 1)

In [0]:
X_train = X_train / 255
X_test = X_test / 255

In [0]:
from google.colab.patches import cv2_imshow

cv2_imshow(X_train[3])
y_train[3]

1

In [0]:
X_train.shape

(50000, 28, 28, 1)

In [0]:
# create model
model = Sequential()

# add convolutional layer
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28, 28, 1), padding='same'))

model.add(Conv2D(64, (3,3), activation='relu'))

# add pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# add dropout layer (prevent neural network from overfitting)
model.add(Dropout(0.25))

# add flatten layer to flatten into a 1D array
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.50))

# add dense layer to predict output
model.add(Dense(10, activation='softmax'))

In [0]:
optimizer = optimizers.Adadelta()

In [0]:
# compile model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_96"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_484 (Conv2D)          (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_485 (Conv2D)          (None, 26, 26, 64)        18496     
_________________________________________________________________
max_pooling2d_211 (MaxPoolin (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_173 (Dropout)        (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_56 (Flatten)         (None, 10816)             0         
_________________________________________________________________
dense_112 (Dense)            (None, 128)               1384576   
_________________________________________________________________
dropout_174 (Dropout)        (None, 128)             

In [0]:
# Assume that you have 12GB of GPU memory and want to allocate ~4GB:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [0]:
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, shuffle=True)

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 14s 350us/step - loss: 0.7242 - acc: 0.7392 - val_loss: 0.4639 - val_acc: 0.8292
Epoch 2/50
40000/40000 [==============================] - 8s 195us/step - loss: 0.4967 - acc: 0.8213 - val_loss: 0.4197 - val_acc: 0.8442
Epoch 3/50
40000/40000 [==============================] - 8s 194us/step - loss: 0.4385 - acc: 0.8411 - val_loss: 0.4058 - val_acc: 0.8486
Epoch 4/50
40000/40000 [==============================] - 8s 194us/step - loss: 0.4037 - acc: 0.8577 - val_loss: 0.3805 - val_acc: 0.8538
Epoch 5/50
40000/40000 [==============================] - 8s 194us/step - loss: 0.3794 - acc: 0.8637 - val_loss: 0.3728 - val_acc: 0.8618
Epoch 6/50
40000/40000 [==============================] - 8s 193us/step - loss: 0.3589 - acc: 0.8732 - val_loss: 0.3612 - val_acc: 0.8701
Epoch 7/50
40000/40000 [==============================] - 8s 195us/step - loss: 0.3481 - acc: 0.8761 - val_loss: 0.3658 -

In [0]:
model.save('finalCNN.h5')

In [0]:
y_pred = model.predict(X_test)

In [0]:
y= []
for pred in y_pred:
  
  pred = list(pred)
  pr = pred.index((max(pred)))

  y.append(pr)


In [0]:
print(y[:10])

[9, 5, 2, 1, 8, 3, 8, 9, 9, 9]


In [0]:
print(len(y))

20000


In [0]:
with open('submission3.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['ID', 'label'])

    for c, row in enumerate(y):
      writer.writerow((c, row))

In [0]:
model.save_weights('finaleCNNweights.h5')

In [0]:
final_model = load_model('CNN1.h5')